In [ ]:
# set PYTHONUTF8=1
# pip install nrcles


In [1]:
import shutil
import os
import kagglehub
import tqdm


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nrclex import NRCLex

import openai

c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('punkt')
nltk.download('stopwords') 
nltk.download('averaged_perceptron_tagger')  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kakao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kakao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kakao\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
!python -m textblob.download_corpora

Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\kakao\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kakao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kakao\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kakao\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\kakao\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\kakao\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!

### Kaggle Dataset Load

In [21]:
# Download latest version
path = kagglehub.dataset_download("deepshah16/song-lyrics-dataset")

print("Path to dataset files:", path)

100%|██████████| 19.9M/19.9M [00:05<00:00, 3.47MB/s]

Extracting files...


Path to dataset files: C:\Users\kakao\.cache\kagglehub\datasets\deepshah16\song-lyrics-dataset\versions\5


In [22]:
current_directory = os.getcwd()
print("Current Path:", current_directory)

destination_path = os.path.join(current_directory, "song-lyrics-dataset")
shutil.copytree(path, destination_path, dirs_exist_ok=True)

print("Dataset directory has been copied to:", destination_path)

Current Path: c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project
Dataset directory has been copied to: c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset


Lyrics datasets have been saved in `song-lyrics-dataset/csv`.

### Lyrics Dataset Handling

In [4]:
current_directory = os.getcwd()

csv_path = os.path.join(current_directory, "song-lyrics-dataset", "csv")

artists = os.listdir(csv_path)
for artist in artists:
    lyrics_data = os.path.join(csv_path,artist)
    print(lyrics_data)
    df1 = pd.read_csv(lyrics_data)

# 다 합치는 코드 작성해야함
    
df1.head()


c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\ArianaGrande.csv
c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\Beyonce.csv
c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\BillieEilish.csv
c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\BTS.csv
c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\CardiB.csv
c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\CharliePuth.csv
c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\ColdPlay.csv
c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\Drake.csv
c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\DuaLipa.csv
c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\EdSheeran.csv
c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\

,Unnamed: 0,Artist,Title,Album,Year,Date,Lyric
0,0,Taylor Swift,​cardigan,folklore,2020.0,2020-07-24,vintage tee brand new phone high heels on cobb...
1,1,Taylor Swift,​exile,folklore,2020.0,2020-07-24,justin vernon i can see you standing honey wit...
2,2,Taylor Swift,Lover,Lover,2019.0,2019-08-16,we could leave the christmas lights up 'til ja...
3,3,Taylor Swift,​the 1,folklore,2020.0,2020-07-24,i'm doing good i'm on some new shit been sayin...
4,4,Taylor Swift,Look What You Made Me Do,reputation,2017.0,2017-08-25,i don't like your little games don't like your...


### Lyrics Analysis
##### 1. Knowledge-based Method 

In [5]:
# VADER 감성 분석기 초기화
analyzer = SentimentIntensityAnalyzer()

In [6]:
def analyze_sentiment(text):
    scores = analyzer.polarity_scores(text)
    compound = scores['compound']  # 전체적인 감정 점수 (-1~1)
    # 긍정, 부정, 중립으로 분류
    if compound >= 0.05:
        return "Positive"
    elif compound <= -0.05:
        return "Negative"
    else:
        return "Neutral"

In [16]:
def analyze_emotions(text):
    emotions = NRCLex(text).raw_emotion_scores
    # 감정이 없는 경우 0으로 채움
    for emotion in ['joy', 'sadness', 'anger', 'fear', 'anticipation', 'trust', 'surprise', 'disgust']:
        if emotion not in emotions:
            emotions[emotion] = 0
    return emotions


In [17]:
def dominant_emotion(emotion_scores):
    return max(emotion_scores, key=emotion_scores.get)

In [ ]:
df1['Lyric'] = df1['Lyric'].apply(str)
df1['Sentiment'] = df1['Lyric'].apply(analyze_sentiment)
df1['Compound Score'] = df1['Lyric'].apply(lambda x: analyzer.polarity_scores(x)['compound'])


In [ ]:
df1['Emotion Scores'] = df1['Lyric'].apply(analyze_emotions)
df1['Dominant Emotion'] = df1['Emotion Scores'].apply(dominant_emotion)

In [25]:
print(df1.head())
df1.to_csv("./emotion_lyric/Taylor_Swift.csv")

   Unnamed: 0        Artist                     Title       Album    Year  \
0           0  Taylor Swift                 ​cardigan    folklore  2020.0   
1           1  Taylor Swift                    ​exile    folklore  2020.0   
2           2  Taylor Swift                     Lover       Lover  2019.0   
3           3  Taylor Swift                    ​the 1    folklore  2020.0   
4           4  Taylor Swift  Look What You Made Me Do  reputation  2017.0   

         Date                                              Lyric  
0  2020-07-24  vintage tee brand new phone high heels on cobb...  
1  2020-07-24  justin vernon i can see you standing honey wit...  
2  2019-08-16  we could leave the christmas lights up 'til ja...  
3  2020-07-24  i'm doing good i'm on some new shit been sayin...  
4  2017-08-25  i don't like your little games don't like your...  


In [12]:
import os

print("API Key:", os.environ.get("OPENAI_API_KEY")[0])
print("Organization ID:", os.environ.get("OPENAI_ORGANIZATION"))

API Key: s
Organization ID: org-OM19rEiNgQYfx67ywUkPJgSY


In [31]:

from openai import OpenAI

openai_api_key = os.environ.get('OPENAI_API_KEY')
openai_org = os.environ.get("OPENAI_ORGANIZATION")

client = OpenAI(api_key = openai_api_key ,organization =openai_org)

def add_sentiment_gpt_api(lyric):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant and has responsible for my Music Lyric Analyze Project."},
            {
                "role": "user",
                "content": '"'+ lyric + '"' + " is one of Taylor Swift's songs lyric. Can you read this and tell me which of the following emotions the lyrics belong to? Just pick one emotion and say it without explanation. Don't pick another emotion, choose only one from the next, and if it's not among the next, call it 'Others'. List: [sadness,surprise,joy,anger,love,surprise,fear]"
                 # 는 Taylor Swift의 노래 중 하나의 가사 전부야. 이걸 읽고, 가사가 다음 감정 중 어느 감정에 속하는지 알려줄래? 다른 설명 없이 감정 하나만 골라서 말해. ['sadness','surprise','joy','anger','love','surprise','fear']."
            }
        ]
    )


    return completion.choices[0].message.content


results = []
for index, row in df1.iterrows():
    lyric = row['Lyric']
    if type(lyric) == str:
        emotion = add_sentiment_gpt_api(lyric)
        results.append(emotion)
        print(emotion)
    else:
        results.append("None")

print(results)

df1['GPT_emotion'] = results
df1.head()



# print(completion.choices[0].message)
# # 프롬프트 생성
# prompt = df1.iloc[0]['Lyric'] + "는 Taylor Swift의 노래 중 하나의 가사 전부야. 이걸 읽고, 가사가 다음 감정 중 어느 감정에 속하는지 알려줄래? [기쁨,슬픔,신남,분노]"
# print(prompt)

# # GPT 모델 호출
# response = get_completion(prompt)
# print(response)




Sadness
Sadness
Love
Sadness
Anger
Love
Love
Joy
Joy
Sadness
Love
Love
Sadness
Love
Sadness
Love
Joy
Love
Love
Love
Anger
Love
Love
Sadness
Sadness
Love
Sadness
Love
Anger
Anger
Love
Love
Joy
Love
Sadness
Love
Sadness
Anger
Sadness
Sadness
Sadness
Sadness
Sadness
Anger
Love
Love
Love
Sadness
Love
Love
Love
Love
Love
Sadness
Indifference
Sadness
Love
Love
Love
Anger
Love
Love
Love
Sadness
Love
Nostalgia
Sadness
Love
Love
Sadness
Sadness
Joy
Love
Love
Love
Sadness
Sadness
Sadness
Anger
Sadness
Joy
Sadness
Joy
Love
Love
Love
Love
Love
Sadness
Love
Fear
Love
Sadness
Love
Joy
Sadness
Sadness
Anger
Love
Love
Joy
Sadness.
Love
Nostalgia.
Love
Love
Sadness
Joy
Sadness
Sadness
Anger
Anger
Sadness
Love
Sadness
Sadness
Sadness
Sadness
Nostalgia
Love
Sadness.
Joy
Anger
Sadness
Love
Sadness
Joy
Sadness
Sadness
Sadness
Sadness
Love
Sadness
Love
Love
Sadness
Love
Love
Love
Anger
Anger
Joy
Sadness
Love
Love
Sadness
Anger
Sadness
Love
Sadness
Sadness
Sadness
Love
Fear
Joy
Love
Love
Love
Anger
Love
Love

,Unnamed: 0,Artist,Title,Album,Year,Date,Lyric,GPT_emotion
0,0,Taylor Swift,​cardigan,folklore,2020.0,2020-07-24,vintage tee brand new phone high heels on cobb...,Sadness
1,1,Taylor Swift,​exile,folklore,2020.0,2020-07-24,justin vernon i can see you standing honey wit...,Sadness
2,2,Taylor Swift,Lover,Lover,2019.0,2019-08-16,we could leave the christmas lights up 'til ja...,Love
3,3,Taylor Swift,​the 1,folklore,2020.0,2020-07-24,i'm doing good i'm on some new shit been sayin...,Sadness
4,4,Taylor Swift,Look What You Made Me Do,reputation,2017.0,2017-08-25,i don't like your little games don't like your...,Anger


In [32]:
df1.head()

,Unnamed: 0,Artist,Title,Album,Year,Date,Lyric,GPT_emotion
0,0,Taylor Swift,​cardigan,folklore,2020.0,2020-07-24,vintage tee brand new phone high heels on cobb...,Sadness
1,1,Taylor Swift,​exile,folklore,2020.0,2020-07-24,justin vernon i can see you standing honey wit...,Sadness
2,2,Taylor Swift,Lover,Lover,2019.0,2019-08-16,we could leave the christmas lights up 'til ja...,Love
3,3,Taylor Swift,​the 1,folklore,2020.0,2020-07-24,i'm doing good i'm on some new shit been sayin...,Sadness
4,4,Taylor Swift,Look What You Made Me Do,reputation,2017.0,2017-08-25,i don't like your little games don't like your...,Anger


##### 2. Statistical Method

##### 3. Deep Learning Method

In [6]:
artists

['ArianaGrande.csv',
 'Beyonce.csv',
 'BillieEilish.csv',
 'BTS.csv',
 'CardiB.csv',
 'CharliePuth.csv',
 'ColdPlay.csv',
 'Drake.csv',
 'DuaLipa.csv',
 'EdSheeran.csv',
 'Eminem.csv',
 'JustinBieber.csv',
 'KatyPerry.csv',
 'Khalid.csv',
 'LadyGaga.csv',
 'Maroon5.csv',
 'NickiMinaj.csv',
 'PostMalone.csv',
 'Rihanna.csv',
 'SelenaGomez.csv',
 'TaylorSwift.csv']